In [ ]:
!nvidia-smi

Sat Nov  7 05:25:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 11.6MB/s 
     |████████████████████████████████| 890kB 59.6MB/s 
     |████████████████████████████████| 2.9MB 51.9MB/s 
     |████████████████████████████████| 1.1MB 55.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8c21a62b14eec67da15a667193339eb759d819f4c914ddfb53c0ec0ba245995c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install sklearn

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [ ]:
df = pd.read_csv("processed_data.csv", encoding = "ISO-8859-1")
df = df.dropna()

In [ ]:
df.head()

,text,label
0,HOUSE RENTAL AGREEMENT,none
1,"Rental Agreement made on Jan 10, 2011, between...",arg_on
2,WHERE IT IS AGREED AND DECLARED AS FOLLOWS:,none
3,The Landlord agrees to let out and the tenant ...,arg_det
4,"The tenant has paid Rs.', '100000(0ne lack Rup...",arg_val


In [ ]:
df['label_code'] = df['label'].map({"arg_det":0,
                                                       "arg_notice":1,
                                                       "arg_on":2,
                                                       "arg_till":3,
                                                       "arg_val":4,
                                                       "none":5
                                                      }).astype("int32")

In [ ]:
df.head()

,text,label,label_code
0,HOUSE RENTAL AGREEMENT,none,5
1,"Rental Agreement made on Jan 10, 2011, between...",arg_on,2
2,WHERE IT IS AGREED AND DECLARED AS FOLLOWS:,none,5
3,The Landlord agrees to let out and the tenant ...,arg_det,0
4,"The tenant has paid Rs.', '100000(0ne lack Rup...",arg_val,4


In [ ]:
# split train dataset into train, validation and test sets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['text'], df['label_code'], 
                                                                    random_state=2018, 
                                                                    test_size=0.4, 
                                                                    stratify=df['label_code'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.4, 
                                                                stratify=temp_labels)

In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = 500,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = 500,
    pad_to_max_length=True,
    truncation=True
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = 500,
    pad_to_max_length=True,
    truncation=True
)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
## convert lists to tensors

train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [ ]:
train_labels.unique()

array([1, 5, 4, 2, 0, 3], dtype=int32)

In [ ]:

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,6)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask)
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),
                  lr = 1e-5)          # learning rate

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print("Class Weights:",class_weights)

Class Weights: [3.07       8.52777778 7.675      8.07894737 3.8375     0.19832041]


In [ ]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 100

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds


In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 100

Evaluating...

Training Loss: 1.787
Validation Loss: 1.783

 Epoch 2 / 100

Evaluating...

Training Loss: 1.778
Validation Loss: 1.776

 Epoch 3 / 100

Evaluating...

Training Loss: 1.760
Validation Loss: 1.770

 Epoch 4 / 100

Evaluating...

Training Loss: 1.758
Validation Loss: 1.762

 Epoch 5 / 100

Evaluating...

Training Loss: 1.747
Validation Loss: 1.755

 Epoch 6 / 100

Evaluating...

Training Loss: 1.739
Validation Loss: 1.750

 Epoch 7 / 100

Evaluating...

Training Loss: 1.729
Validation Loss: 1.748

 Epoch 8 / 100

Evaluating...

Training Loss: 1.728
Validation Loss: 1.743

 Epoch 9 / 100

Evaluating...

Training Loss: 1.725
Validation Loss: 1.743

 Epoch 10 / 100

Evaluating...

Training Loss: 1.718
Validation Loss: 1.745

 Epoch 11 / 100

Evaluating...

Training Loss: 1.724
Validation Loss: 1.738

 Epoch 12 / 100

Evaluating...

Training Loss: 1.695
Validation Loss: 1.738

 Epoch 13 / 100

Evaluating...

Training Loss: 1.701
Validation Loss: 1.734

 Epoch 

KeyboardInterrupt: ignored

In [ ]:

#load weights
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [ ]:
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.13      0.46      0.20        13
           1       0.04      0.20      0.07         5
           2       0.32      1.00      0.48         6
           3       0.15      0.40      0.22         5
           4       0.09      0.70      0.15        10
           5       0.92      0.28      0.43       207

    accuracy                           0.33       246
   macro avg       0.27      0.51      0.26       246
weighted avg       0.80      0.33      0.40       246



In [ ]:
test_seq[1]

tensor([  101,  2026,  3372,  2527,  1029,  1055,  3330,  2136, 16473,  1996,
         2974,  4132,  2008,  7861, 11452,  2015,  2256,  6304,  1029,  6023,
         3325,  1998, 12939,  1996,  5744,  4834,  1997,  3688,  2013, 20141,
         2000,  2256,  6304,  1029, 26581,  2015,  1012,  2057,  2147,  2006,
         2752,  2107,  2004,  2311,  5294,  1011,  4094,  4773,  1011,  5097,
         1010, 11973,  5310,  1011, 19706,  1010,  2502,  1011,  2951, 25095,
         1010,  4684, 18726,  1010,  2147, 12314,  3001,  1010, 12612,  1011,
         2968,  4385,  1012,  2057,  2024,  1037,  2235,  2974,  2136,  2073,
         2169,  3265,  2038,  1037,  4121,  4254,  1012,  2017,  2097,  2031,
         1996,  4495,  2000,  2022,  2112,  1997,  1037,  5901,  3652,  3029,
         1998,  5114,  7524,  2000,  2035,  1996,  3033,  1997,  1037,  7721,
        17338, 15810,  3401,  4132,  1012,  2017,  2097,  2022,  1037,  2112,
         1997,  1024,  4425,  4677,  2968,  1006,  8040,  2213, 

In [ ]:
text_str = """THIS DEED OF RENTAL AGREEMENT executed at Chennai this 19th day of August 2013 between:"""

In [ ]:

text = [text_str]

# encode text
sent_id = tokenizer.batch_encode_plus(text, max_length = 500,pad_to_max_length=True,truncation=True)

# output
# print(sent_id['input_ids'][0])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
check_seq = torch.tensor(sent_id['input_ids'])
check_mask = torch.tensor(sent_id['attention_mask'])
check_preds = model(check_seq.to(device), check_mask.to(device))
check_preds = check_preds.detach().cpu().numpy()


In [ ]:
check_preds

array([[-1.7328563, -1.91032  , -1.4678383, -1.9384296, -1.8873948,
        -1.9013317]], dtype=float32)

In [ ]:
check_preds = np.argmax(check_preds, axis = 1)

In [ ]:
check_preds

array([2])

In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=820feafa1ed075e4cf69169becf60a497c40599eb68a09803538974e92163123
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
url = 'https://googledrive.com/host/1T2cku8oxYeho6m-VI8uqvfkukqLBMD6g/saved_weights.pt'
wget.download(url)

HTTPError: ignored